# Setup

In [ ]:
!pip install -q kaggle
!pip install -q ydata_profiling
!pip install -q imbalanced-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 398.7/398.7 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.7/679.7 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 2.1 MB/s eta 0:00:00


In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"saatvikmtech","key":"e675c36acec415887dcabdbf052957ab"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d gauravduttakiit/smoker-status-prediction-using-biosignals
!unzip smoker-status-prediction-using-biosignals.zip

Dataset URL: https://www.kaggle.com/datasets/gauravduttakiit/smoker-status-prediction-using-biosignals
License(s): apache-2.0
  0% 0.00/1.30M [00:00<?, ?B/s]
100% 1.30M/1.30M [00:00<00:00, 577MB/s]
Archive:  smoker-status-prediction-using-biosignals.zip
  inflating: test_dataset.csv        
  inflating: train_dataset.csv       


In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport

In [ ]:
df = pd.read_csv('train_dataset.csv')
df

,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,relaxation,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
0,35,170,85,97.0,0.9,0.9,1,1,118,78,...,70,142,19.8,1,1.0,61,115,125,1,1
1,20,175,110,110.0,0.7,0.9,1,1,119,79,...,71,114,15.9,1,1.1,19,25,30,1,0
2,45,155,65,86.0,0.9,0.9,1,1,110,80,...,57,112,13.7,3,0.6,1090,1400,276,0,0
3,45,165,80,94.0,0.8,0.7,1,1,158,88,...,46,91,16.9,1,0.9,32,36,36,0,0
4,20,165,60,81.0,1.5,0.1,1,1,109,64,...,47,92,14.9,1,1.2,26,28,15,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38979,40,165,60,80.0,0.4,0.6,1,1,107,60,...,61,72,12.3,1,0.5,18,18,21,1,0
38980,45,155,55,75.0,1.5,1.2,1,1,126,72,...,76,131,12.5,2,0.6,23,11,12,0,0
38981,40,170,105,124.0,0.6,0.5,1,1,141,85,...,48,138,17.1,1,0.8,24,23,35,1,1
38982,40,160,55,75.0,1.5,1.5,1,1,95,69,...,79,116,12.0,1,0.6,24,20,17,0,1


# Preprocessing

In [ ]:
print(df.duplicated().sum())
df.drop_duplicates(inplace=True)

0


In [ ]:
x = df.drop(columns=['waist(cm)', 'systolic', 'smoking'])
y = df['smoking']

In [ ]:
x[['hearing(left)', 'hearing(right)']] = x[['hearing(left)','hearing(right)']] - 1

x = pd.get_dummies(x, columns=['Urine protein'], prefix='urine_protein')

In [ ]:
x

,age,height(cm),weight(kg),eyesight(left),eyesight(right),hearing(left),hearing(right),relaxation,fasting blood sugar,Cholesterol,...,AST,ALT,Gtp,dental caries,urine_protein_1,urine_protein_2,urine_protein_3,urine_protein_4,urine_protein_5,urine_protein_6
0,35,170,85,0.9,0.9,0,0,78,97,239,...,61,115,125,1,True,False,False,False,False,False
1,20,175,110,0.7,0.9,0,0,79,88,211,...,19,25,30,1,True,False,False,False,False,False
2,45,155,65,0.9,0.9,0,0,80,80,193,...,1090,1400,276,0,False,False,True,False,False,False
3,45,165,80,0.8,0.7,0,0,88,249,210,...,32,36,36,0,True,False,False,False,False,False
4,20,165,60,1.5,0.1,0,0,64,100,179,...,26,28,15,0,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38978,40,170,65,1.5,1.5,0,0,62,91,187,...,28,43,36,1,True,False,False,False,False,False
38979,40,165,60,0.4,0.6,0,0,60,93,144,...,18,18,21,1,True,False,False,False,False,False
38980,45,155,55,1.5,1.2,0,0,72,91,227,...,23,11,12,0,False,True,False,False,False,False
38982,40,160,55,1.5,1.5,0,0,69,102,206,...,24,20,17,0,True,False,False,False,False,False


# Models

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

In [ ]:
categorical_cols = ['hearing(left)', 'hearing(right)', 'Urine protein', 'dental caries', 'smoking']
continuous_cols = [col for col in x.columns if col not in categorical_cols]

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
pp = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), continuous_cols)
    ],
    remainder='passthrough'
)

## Logistic Regression

In [ ]:
lrpipe = Pipeline([
    ('preprocess', pp),
    ('clf', LogisticRegression(max_iter=10000))
])

lrparam = {
    'clf__penalty': [None, 'elasticnet', 'l1', 'l2', ],
    'clf__C': [0.1, 0.3, 1, 3, 10, 30],
    'clf__l1_ratio': [0.25, 0.5, 0.75],
    'clf__solver': ['saga', 'lbfgs']
}

In [ ]:
gridlr = GridSearchCV(estimator=lrpipe, param_grid=lrparam, scoring='accuracy', cv=skf, verbose=3, return_train_score=True)
gridlr.fit(x, y)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1196: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=None)
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1207: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


[CV 1/5] END clf__C=0.1, clf__l1_ratio=0.25, clf__penalty=None, clf__solver=saga;, score=(train=0.720, test=0.724) total time=   5.1s


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1196: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=None)
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1207: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


[CV 2/5] END clf__C=0.1, clf__l1_ratio=0.25, clf__penalty=None, clf__solver=saga;, score=(train=0.723, test=0.717) total time=   4.1s


...

[CV 4/5] END clf__C=30, clf__l1_ratio=0.75, clf__penalty=l2, clf__solver=lbfgs;, score=(train=0.725, test=0.710) total time=   0.3s


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1196: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(


[CV 5/5] END clf__C=30, clf__l1_ratio=0.75, clf__penalty=l2, clf__solver=lbfgs;, score=(train=0.720, test=0.726) total time=   0.3s


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
180 fits failed out of a total of 720.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
90 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py", line 662, in fit
    self._final_estimator.fit(Xt, y, **

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['age',
                                                                          'height(cm)',
                                                                          'weight(kg)',
                                                                          'eyesight(left)',
                                                                          'eyesight(right)',
                                                                          'relaxation',
                                                                          'fasting '
                                                                          'blood '
                                                                          'sugar',
                                                                          'Cholesterol',
                                                                          'triglyceride',
                                                                          'HDL',
                                                                          'LDL',
                                                                          'hemoglobin...
                                                                          'urine_protein_1',
                                                                          'urine_protein_2',
                                                                          'urine_protein_3',
                                                                          'urine_protein_4',
                                                                          'urine_protein_5',
                                                                          'urine_protein_6'])])),
                                       ('clf',
                                        LogisticRegression(max_iter=10000))]),
             param_grid={'clf__C': [0.1, 0.3, 1, 3, 10, 30],
                         'clf__l1_ratio': [0.25, 0.5, 0.75],
                         'clf__penalty': [None, 'elasticnet', 'l1', 'l2'],
                         'clf__solver': ['saga', 'lbfgs']},
             return_train_score=True, scoring='accuracy', verbose=3)

In [ ]:
print(gridlr.best_score_)
print(gridlr.best_params_)

bestlr = gridlr.best_estimator_
ypred = bestlr.predict(x)
print(accuracy_score(y, ypred))
print(classification_report(y, ypred))
print(confusion_matrix(y, ypred))

0.7211283000120841
{'clf__C': 1, 'clf__l1_ratio': 0.25, 'clf__penalty': 'l1', 'clf__solver': 'saga'}
0.7210087548928795
              precision    recall  f1-score   support

           0       0.76      0.81      0.79     21209
           1       0.63      0.56      0.60     12258

    accuracy                           0.72     33467
   macro avg       0.70      0.69      0.69     33467
weighted avg       0.72      0.72      0.72     33467

[[17257  3952]
 [ 5385  6873]]


## Support Vector Classifier

In [ ]:
svcpipe = Pipeline([
    ('preprocess', pp),
    ('clf', SVC(max_iter=100000, kernel='rbf'))
])

svcparam = {
    'clf__C': [0.1, 0.3, 1, 3, 10]
}

In [ ]:
gridsvc = GridSearchCV(estimator=svcpipe, param_grid=svcparam, scoring='accuracy', cv=skf, verbose=3, return_train_score=True)
gridsvc.fit(x, y)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 1/5] END ....clf__C=0.1;, score=(train=0.745, test=0.745) total time=  43.0s
[CV 2/5] END ....clf__C=0.1;, score=(train=0.745, test=0.736) total time=  40.7s
[CV 3/5] END ....clf__C=0.1;, score=(train=0.742, test=0.748) total time=  40.8s
[CV 4/5] END ....clf__C=0.1;, score=(train=0.747, test=0.730) total time=  39.1s
[CV 5/5] END ....clf__C=0.1;, score=(train=0.746, test=0.736) total time=  40.8s
[CV 1/5] END ....clf__C=0.3;, score=(train=0.756, test=0.749) total time=  39.5s
[CV 2/5] END ....clf__C=0.3;, score=(train=0.756, test=0.742) total time=  39.6s
[CV 3/5] END ....clf__C=0.3;, score=(train=0.753, test=0.753) total time=  39.3s
[CV 4/5] END ....clf__C=0.3;, score=(train=0.759, test=0.736) total time=  39.0s
[CV 5/5] END ....clf__C=0.3;, score=(train=0.757, test=0.741) total time=  39.9s
[CV 1/5] END ......clf__C=1;, score=(train=0.767, test=0.753) total time=  39.2s
[CV 2/5] END ......clf__C=1;, score=(train=0.771,

/usr/local/lib/python3.12/dist-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['age',
                                                                          'height(cm)',
                                                                          'weight(kg)',
                                                                          'eyesight(left)',
                                                                          'eyesight(right)',
                                                                          'relaxation',
                                                                          'fasting '
                                                                          'blood '
                                                                          'sugar',
                                                                          'Cholesterol',
                                                                          'triglyceride',
                                                                          'HDL',
                                                                          'LDL',
                                                                          'hemoglobin',
                                                                          'serum '
                                                                          'creatinine',
                                                                          'AST',
                                                                          'ALT',
                                                                          'Gtp',
                                                                          'urine_protein_1',
                                                                          'urine_protein_2',
                                                                          'urine_protein_3',
                                                                          'urine_protein_4',
                                                                          'urine_protein_5',
                                                                          'urine_protein_6'])])),
                                       ('clf', SVC(max_iter=100000))]),
             param_grid={'clf__C': [0.1, 0.3, 1, 3, 10]},
             return_train_score=True, scoring='accuracy', verbose=3)

In [ ]:
print(gridsvc.best_score_)
print(gridsvc.best_params_)

bestsvc = gridsvc.best_estimator_
ypred = bestsvc.predict(x)
print(accuracy_score(y, ypred))
print(classification_report(y, ypred))
print(confusion_matrix(y, ypred))

0.7480801372374163
{'clf__C': 1}
0.7690262049182777
              precision    recall  f1-score   support

           0       0.81      0.83      0.82     21209
           1       0.69      0.66      0.68     12258

    accuracy                           0.77     33467
   macro avg       0.75      0.75      0.75     33467
weighted avg       0.77      0.77      0.77     33467

[[17633  3576]
 [ 4154  8104]]


## Dense Neural Network

In [ ]:
nnskf = StratifiedKFold(n_splits=3, shuffle=False)

nnpipe = Pipeline([
    ('preprocess', pp),
    ('clf', MLPClassifier(solver='adam', batch_size=8192, tol=0.1, early_stopping=True, random_state=42, validation_fraction=0.25))
])

nnparam = {
    'clf__learning_rate_init': [0.001, 0.003, 0.01, 0.03, 0.1, 0.3],
    'clf__alpha': [0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 3],
    'clf__hidden_layer_sizes': [(64,), (128,), (64, 64), (128, 128)],
    'clf__activation': ['logistic', 'tanh', 'relu'],
}

In [ ]:
gridnn = GridSearchCV(estimator=nnpipe, param_grid=nnparam, cv=nnskf)
gridnn.fit(x, y)

/usr/local/lib/python3.12/dist-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['age',
                                                                          'height(cm)',
                                                                          'weight(kg)',
                                                                          'eyesight(left)',
                                                                          'eyesight(right)',
                                                                          'relaxation',
                                                                          'fasting '
                                                                          'blood '
                                                                          'sugar',
                                                                          'Cholesterol',
                                                                          'triglyceride',
                                                                          'HDL',
                                                                          'LDL',
                                                                          'hemoglo...
                                                                          'urine_protein_6'])])),
                                       ('clf',
                                        MLPClassifier(batch_size=8192,
                                                      early_stopping=True,
                                                      random_state=42, tol=0.1,
                                                      validation_fraction=0.25))]),
             param_grid={'clf__activation': ['logistic', 'tanh', 'relu'],
                         'clf__alpha': [0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1,
                                        3],
                         'clf__hidden_layer_sizes': [(64,), (128,), (64, 64),
                                                     (128, 128)],
                         'clf__learning_rate_init': [0.001, 0.003, 0.01, 0.03,
                                                     0.1, 0.3]})

In [ ]:
print(gridnn.best_score_)
print(gridnn.best_params_)

bestnn = gridnn.best_estimator_
ypred = bestnn.predict(x)
print(accuracy_score(y, ypred))
print(classification_report(y, ypred))
print(confusion_matrix(y, ypred))

0.7416559430693365
{'clf__activation': 'relu', 'clf__alpha': 0.1, 'clf__hidden_layer_sizes': (128,), 'clf__learning_rate_init': 0.03}
0.7484686407505902
              precision    recall  f1-score   support

           0       0.79      0.83      0.81     21209
           1       0.67      0.61      0.64     12258

    accuracy                           0.75     33467
   macro avg       0.73      0.72      0.72     33467
weighted avg       0.74      0.75      0.75     33467

[[17608  3601]
 [ 4817  7441]]
